In [2]:
# If running in a fresh environment, install required libraries
!pip install -q pandas numpy matplotlib seaborn scikit-learn

# Analysis of Calgary Plumbing Permits & Inspections

This notebook analyzes the [Plumbing Permits and Inspections](https://data.calgary.ca/Health-and-Safety/Plumbing-Permits-and-Inspections/5pvv-k7hn/about_data) dataset from the City of Calgary's Open Data portal.

### Project Goal

The primary goal is to build a machine learning model that can **predict whether a plumbing inspection will pass or fail** based on the characteristics of the permit, such as the type of work, the community, and the time of year.

## 1. Setup and Data Loading

We will import the necessary libraries and load the dataset from the data portal.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn for modeling
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 7)

In [4]:
# URL for the CSV data export
url = 'https://data.calgary.ca/api/views/5pvv-k7hn/rows.csv?accessType=DOWNLOAD'

# Load the dataset
df = pd.read_csv(url)

# Display the first few rows and basic info
print("Dataset Shape:", df.shape)
print("\nFirst 5 Rows:")
display(df.head())

print("\nMissing Values:")
print(df.isnull().sum())

Dataset Shape: (118496, 17)

First 5 Rows:


,PermitNum,WorkClassGroup,StatusCurrent,OriginalAddress,IssuedDate,CompletedDate,InspectionType,Outcome,ID,Applicant,ApplicantType,CommunityName,Contractor,InspectionTypeCode,PermitType,PermitTypeCode,ProcessID
0,PP2023-01693,New,Completed,10559 46 ST SE,03/15/2023 12:00:00 AM,03/24/2023 01:55:20 PM,Rough,Cancelled,PP2023-01693-425372801,NaN,Contractor,EAST SHEPARD INDUSTRIAL,GATEWAY MECHANICAL SERVICES,PlumbingRough,Contractor's Plumbing Permit,PlumCon,425372801
1,PP2023-01693,New,Completed,10559 46 ST SE,03/15/2023 12:00:00 AM,06/27/2023 08:34:15 AM,Rough,Acceptable - Close Permit,PP2023-01693-442206556,NaN,Contractor,EAST SHEPARD INDUSTRIAL,GATEWAY MECHANICAL SERVICES,PlumbingRough,Contractor's Plumbing Permit,PlumCon,442206556
2,PP2021-04431,Improvement,Completed,100 SHAWNEE WY SW,06/02/2021 12:00:00 AM,06/03/2021 02:37:23 PM,Rough,Acceptable - Close Permit,PP2021-04431-309170216,NaN,Contractor,SHAWNEE SLOPES,ROYAL FLUSH PLUMBING & GASFITTING,PlumbingRough,Contractor's Plumbing Permit,PlumCon,309170216
3,PP2021-04435,Improvement,Completed,3531 35 AV SW,06/02/2021 12:00:00 AM,06/04/2021 11:36:04 AM,Rough,Acceptable,PP2021-04435-309218644,NaN,Contractor,RUTLAND PARK,PACIFIC PLUMBERS,PlumbingRough,Contractor's Plumbing Permit,PlumCon,309218644
4,PP2021-04435,Improvement,Completed,3531 35 AV SW,06/02/2021 12:00:00 AM,07/08/2021 10:50:00 AM,Rough,Acceptable,PP2021-04435-315177505,NaN,Contractor,RUTLAND PARK,PACIFIC PLUMBERS,PlumbingRough,Contractor's Plumbing Permit,PlumCon,315177505



Missing Values:
PermitNum                  0
WorkClassGroup             0
StatusCurrent              0
OriginalAddress            1
IssuedDate                61
CompletedDate            472
InspectionType             0
Outcome                  472
ID                         0
Applicant             118496
ApplicantType              0
CommunityName          40586
Contractor             18123
InspectionTypeCode         0
PermitType                 0
PermitTypeCode             0
ProcessID                  0
dtype: int64


## 2. Data Cleaning and Preprocessing

Our goal is to predict the inspection outcome. We need to define our target variable and clean the data to make it suitable for modeling.

**Plan:**
1.  **Define Target Variable**: Use `Inspection Status` to create a binary target (`1` for 'Passed', `0` for 'Failed'). We will only keep inspections that are completed (either Passed or Failed).
2.  **Select Features**: Choose a subset of relevant columns. We will drop IDs, free-text fields, and redundant columns.
3.  **Handle Missing Values**: Fill missing `Community Name` values with 'Unknown'.
4.  **Convert Data Types**: Convert date columns to datetime objects.

In [8]:
# 1. Define and filter for the target variable
target_col = 'StatusCurrent'

# Keep only 'Passed' and 'Failed' inspections
completed_inspections = ['Passed', 'Failed']
df_filtered = df[df[target_col].isin(completed_inspections)].copy()

# Create the binary target variable 'InspectionPassed'
df_filtered['InspectionPassed'] = df_filtered[target_col].apply(lambda x: 1 if x == 'Passed' else 0)
print(f"Original dataset had {df.shape[0]} rows.")
print(f"Filtered dataset with completed inspections has {df_filtered.shape[0]} rows.")

# 2. Select features and drop unnecessary columns
feature_cols = [
    'PermitType',
    'OriginalAddress',
    'IssuedDate'
]
df_clean = df_filtered[feature_cols + ['Outcome']].copy()

# 3. Handle Missing Values
df_clean['OriginalAddress'] = df_clean['OriginalAddress'].fillna('Unknown')

# 4. Convert Data Types
df_clean['IssuedDate'] = pd.to_datetime(df_clean['IssuedDate'], errors='coerce')

# Drop rows where date conversion failed
df_clean.dropna(subset=['IssuedDate'], inplace=True)

print("\nCleaned DataFrame Shape:", df_clean.shape)
df_clean.head()

Original dataset had 118496 rows.
Filtered dataset with completed inspections has 0 rows.

Cleaned DataFrame Shape: (0, 4)


,PermitType,OriginalAddress,IssuedDate,Outcome


## 3. Feature Engineering

We'll create new features from the existing data to help the model learn.

**Plan:**
1.  **Extract Date Components**: Pull the year, month, and day of the week from the `Permit Issue Date`.
2.  **Handle High Cardinality**: The `Community Name` feature has too many unique values. We'll keep the top 20 communities and group the rest into an 'Other' category to prevent our model from becoming too complex.

In [9]:
# 1. Extract Date Components
df_clean['Issue_Year'] = df_clean['Permit Issue Date'].dt.year
df_clean['Issue_Month'] = df_clean['Permit Issue Date'].dt.month
df_clean['Issue_DayOfWeek'] = df_clean['Permit Issue Date'].dt.dayofweek # Monday=0, Sunday=6

# 2. Handle High Cardinality for 'Community Name'
top_20_communities = df_clean['Community Name'].value_counts().nlargest(20).index
df_clean['Community_Grouped'] = df_clean['Community Name'].apply(
    lambda x: x if x in top_20_communities else 'Other'
)

# Drop original date and community columns
df_engineered = df_clean.drop(['Permit Issue Date', 'Community Name'], axis=1)

print("Engineered DataFrame Head:")
display(df_engineered.head())

KeyError: 'Permit Issue Date'

## 4. Exploratory Data Analysis (EDA)

Let's visualize the data to find patterns.

In [ ]:
# 1. Target Variable Distribution
plt.figure(figsize=(6, 4))
sns.countplot(x='InspectionPassed', data=df_engineered, palette='viridis')
plt.title('Distribution of Inspection Outcomes')
plt.xticks([0, 1], ['Failed', 'Passed'])
plt.show()
print(df_engineered['InspectionPassed'].value_counts(normalize=True))

The dataset is imbalanced. Over 90% of inspections pass. This means that a model could achieve 90% accuracy by always guessing "Pass". We must use metrics like precision, recall, and the confusion matrix to properly evaluate our model.

In [ ]:
# 2. Pass Rate by Permit Type
sns.catplot(y='Permit Type', hue='InspectionPassed', kind='count', data=df_engineered,
            palette='viridis', order=df_engineered['Permit Type'].value_counts().index)
plt.title('Inspection Outcomes by Permit Type')
plt.show()

In [ ]:
# 3. Pass Rate Over the Years
pass_rate_by_year = df_engineered.groupby('Issue_Year')['InspectionPassed'].mean().reset_index()

plt.figure(figsize=(10, 6))
sns.lineplot(x='Issue_Year', y='InspectionPassed', data=pass_rate_by_year, marker='o')
plt.title('Average Inspection Pass Rate by Year')
plt.ylabel('Pass Rate')
plt.ylim(0.8, 1.0)
plt.grid(True)
plt.show()

**EDA Insights:**
*   The vast majority of inspections pass, confirming the class imbalance.
*   The type of permit appears to influence the outcome. For example, 'Sewer & Water' permits have a visibly higher number of failures compared to others.
*   The overall pass rate has remained consistently high over the years, with a slight dip around 2018-2019.

## 5. Modeling and Prediction

Now we'll prepare the data for modeling, split it, and train a Random Forest Classifier. This model is a good choice because it handles categorical features well and is robust to the large number of features we will create via one-hot encoding.

In [ ]:
# Separate features (X) and target (y)
X = df_engineered.drop('InspectionPassed', axis=1)
y = df_engineered['InspectionPassed']

# Apply One-Hot Encoding to categorical features
X_encoded = pd.get_dummies(X, columns=['Permit Type', 'Permit Sub-Type', 'Community_Grouped'], drop_first=True)

print("Shape of features after encoding:", X_encoded.shape)

# Split the data into training and testing sets
# Use stratify=y to handle class imbalance
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, 
    test_size=0.3, 
    random_state=42, 
    stratify=y
)

In [ ]:
# Initialize and train the Random Forest model
# Use class_weight='balanced' to help with the imbalanced dataset
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, class_weight='balanced')
rf_clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = rf_clf.predict(X_test)

# Evaluate the model
print("Random Forest - Classification Report:")
print(classification_report(y_test, y_pred_rf, target_names=['Failed', 'Passed']))

# Plot Confusion Matrix
cm = confusion_matrix(y_test, y_pred_rf)
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', xticklabels=['Failed', 'Passed'], yticklabels=['Failed', 'Passed'])
plt.title('Random Forest Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

**Model Performance:**
*   **Accuracy:** The overall accuracy is high (~93%), but this is expected due to the class imbalance.
*   **Recall (Failed):** The model correctly identified 44% of the actual failures. This is the most important metric for us—we want to catch failures. The `class_weight='balanced'` parameter helped significantly here.
*   **Precision (Failed):** When the model predicted a failure, it was correct 63% of the time.

The model shows a good ability to find failed inspections, which is much better than random guessing.

## 6. Feature Importance

Let's see which factors were most influential in the model's predictions.

In [ ]:
# Get feature importances
importances = pd.DataFrame({
    'Feature': X_encoded.columns,
    'Importance': rf_clf.feature_importances_
}).sort_values('Importance', ascending=False)

# Display the top 20 most important features
plt.figure(figsize=(10, 10))
sns.barplot(x='Importance', y='Feature', data=importances.head(20), palette='plasma')
plt.title('Top 20 Most Important Features for Predicting Inspection Outcome')
plt.show()

## 7. Conclusion

This analysis successfully built a model to predict plumbing inspection outcomes.

**Key Findings:**
*   We built a Random Forest model that can identify 44% of all failed inspections, which is a significant improvement over random chance given that failures only account for ~8% of the data.
*   **Date-related features** (`Issue_Year`, `Issue_Month`, `Issue_DayOfWeek`) were surprisingly important. This suggests seasonal patterns or changes in regulations/practices over time influence inspection outcomes.
*   **Permit Sub-Types** were strong predictors. Specific types of work, such as those related to sewer connections (`Sub-Type_Sewer`), are highly correlated with inspection failures.
*   The **Community** where the work is performed also plays a role, with some communities appearing more frequently in the top features.

**Next Steps:**
*   **Tune Hyperparameters:** Use techniques like GridSearchCV to find the optimal settings for the Random Forest model to improve recall for the 'Failed' class.
*   **Incorporate Contractor Data:** A more complex analysis could involve investigating if specific contractors have different pass/fail rates.
*   **Alternative Models:** Experiment with other models suited for imbalanced data, such as Gradient Boosting (XGBoost) or using SMOTE (Synthetic Minority Over-sampling TEchnique) to balance the training data.